# Week 3 - Bayesian Optimization

Generate optimized recommendations for Week 3 using utility modules.

## Setup

In [ ]:
import numpy as np
import sys
sys.path.append('..')  # Add parent directory to path

# Import utility modules
from utils.bayesian_optimization import propose_next_point, get_strategy
from utils.data_utils import (
    load_week_data,
    save_week_data,
    combine_with_week_results, 
    print_data_summary
)
from utils.visualization import visualize_all_functions

## 1. Load Week 2 Data

Load the combined data from Week 2 (includes initial + Week 1 results)

In [ ]:
# Load Week 2 clean data (initial + Week 1)
inputs, outputs = load_week_data("../week 2/week2_clean_data.npz")
print_data_summary(inputs, outputs, "Week 2 Data (Initial + Week 1)")

## 2. Add Week 2 Results

**TODO: Update with actual Week 2 results when you receive them**

In [ ]:
# Week 2 submitted points (from Week 2 recommendations)
week2_inputs = {
    1: np.array([0.40, 0.67]),  # TODO: Replace with actual submitted values
    2: np.array([0.70, 0.13]),
    3: np.array([0.99, 1.00, 0.41]),
    4: np.array([0.43, 0.39, 0.43, 0.38]),
    5: np.array([0.38, 0.30, 1.00, 1.00]),
    6: np.array([0.76, 0.27, 0.64, 0.67, 0.16]),
    7: np.array([0.00, 0.34, 0.71, 0.25, 0.41, 0.78]),
    8: np.array([0.04, 0.14, 0.00, 0.04, 1.00, 0.09, 0.00, 0.95])
}

# Week 2 outputs (received from function evaluations)
# TODO: Replace with actual results when you get them
week2_outputs = {
    1: 0.0,  # Replace with actual output
    2: 0.0,
    3: 0.0,
    4: 0.0,
    5: 0.0,
    6: 0.0,
    7: 0.0,
    8: 0.0
}

# Combine with Week 2
inputs, outputs = combine_with_week_results(inputs, outputs, week2_inputs, week2_outputs)
print_data_summary(inputs, outputs, "After Week 2")

## 3. Save Week 3 Clean Data

In [ ]:
# Save combined data for Week 4
save_week_data(inputs, outputs, "week3_clean_data.npz")

## 4. Generate Week 3 Recommendations

In [ ]:
print("=" * 70)
print("WEEK 3 BAYESIAN OPTIMIZATION RECOMMENDATIONS")
print("=" * 70)

week3_recommendations = {}

for func_id in range(1, 9):
    X = inputs[func_id]
    y = outputs[func_id]
    dim = X.shape[1]
    
    # Bounds: all inputs in [0, 1]
    bounds = np.array([[0, 1]] * dim)
    
    # Get strategy (can be customized based on Week 2 results)
    strategy = get_strategy(func_id, week_number=3)
    
    # Get recommendation
    next_point, gp = propose_next_point(
        X, y, bounds,
        acq_func=strategy['acq_func'],
        xi=strategy['xi'],
        kappa=strategy['kappa'],
        n_restarts=50
    )
    
    week3_recommendations[func_id] = next_point
    
    # Display results
    current_best_idx = np.argmax(y)
    current_best = y[current_best_idx]
    
    print(f"\n{'='*70}")
    print(f"FUNCTION {func_id} ({dim}D)")
    print(f"{'='*70}")
    print(f"Data points: {len(y)}")
    print(f"Current best: {current_best:.6f}")
    print(f"Strategy: {strategy['acq_func']} (xi={strategy['xi']}, kappa={strategy['kappa']})")
    print(f"Week 3 recommended point: {next_point}")
    pred_mean = gp.predict(next_point.reshape(1, -1))[0]
    pred_std = gp.predict(next_point.reshape(1, -1), return_std=True)[1][0]
    print(f"Predicted: {pred_mean:.6f} ± {pred_std:.6f}")

print(f"\n{'='*70}")
print("SUMMARY - Week 3 Points to Submit:")
print(f"{'='*70}")
for func_id in range(1, 9):
    point = week3_recommendations[func_id]
    print(f"Function {func_id}: {' – '.join([f'{x:.2f}' for x in point])}")

## 5. Visualizations

In [ ]:
# Generate all visualizations
visualize_all_functions(inputs, outputs, week3_recommendations)